# Лабораторная работа: Распознавание изображений

## Задача: Создать сервис, который будет распознавать изображения багажа

## Датасет: Пять видов багажа. Рюкзаки (Backpack), Сумки (Bag), Портфели (Briefcase), Ручные сумки (Handbag), Чемоданы (Suitcase)

Custom Vision Service - это инструмент для создания кастомных классификаторов изображений. С помощью подготовленного датасета и библиотеки для работы с Custom Vision для Python создать сервис для распознавания изображений багажа. 

### Шаг 1. Импортирование библиотек и нужных инструментов из них

Первым делом нам необходимо импортировать нужные нам инструменты из библиотеки для работы с Custom Vision

Импортировать мы будем два инструмента - для тренировки модели и для работы с ней. 

Сперва импортируем инструменты для тренировки модели

In [2]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

Потом импортируем инструменты для работы с обученной моделью

In [4]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

### Шаг 2. Инициализация трейнера и предиктора.

Два основных объекта, с которыми мы будем работать - трейнер и предиктор. Первый поможет нам обучить модель, а второй поможет работать с моделью напрямую. Для работы с трейнером и предиктором нам нужны ключи для подключения к нашему сервису, развернутому на Azure.

С помощью этих ключей мы сможем подключиться к сервису и получить возможность работать с моделью

In [7]:
training_key = ""
prediction_key = ""

Проинициализируем наши трейнер и предиктор.

In [8]:
trainer = training_api.TrainingApi(training_key)
predictor = prediction_endpoint.PredictionEndpoint(prediction_key)

Создадим проект

In [9]:
project = trainer.create_project("AI_Weekend")

### Шаг 3. Подготовка тегов и загрузка данных.

Поскольку объектов в нашем датасете много, нам необходимо для каждой фотографии подобрать подходящий ей тег. Тегом будет служить название вида багажа. Датасет с багажом будет находиться в папке ClearDataSet.

Создадим несколько тегов.

In [10]:
Backpack_tag = trainer.create_tag(project.id, "Backpack")
Bag_tag = trainer.create_tag(project.id, "Bag")
Briefcase_tag = trainer.create_tag(project.id, "Briefcase")
Handbag_tag = trainer.create_tag(project.id, "Handbag")
Suitcase_tag = trainer.create_tag(project.id, "Suitcase")

Теперь мы готовы загрузить наш размеченный датасет. Теперь нам понадобится библиотека для работы с файловой системой вашей машины.

In [11]:
import os

Для удобства разметим словарь, который мы сможем однозначно перебрать, чтобы правильно загрузить наши изображения на сервис. Ключ будет являться путем к папке, а значение - id для тега.

In [12]:
rootdir = "..\\ClearDataSet\\" ##путь к папке с изображениями

dir_n_tag_dict = {
    "Backpack" : Backpack_tag.id,
    "Bag" : Bag_tag.id,
    "Briefcase" : Briefcase_tag.id,
    "Handbag" : Handbag_tag.id,
    "Suitcase" : Suitcase_tag.id
}

Теперь обойдем папки с изображениями. Сделаем два простых цикла, вложенных один в другой. Попутно будем загружать наши изображения на сервис под нужный нам тег.

In [13]:
for key in dir_n_tag_dict:
    for image in os.listdir(os.fsencode(rootdir + key)):
        with open(rootdir + key + "\\" + os.fsdecode(image), mode="rb") as img_data:
            trainer.create_images_from_data(project.id, img_data, [dir_n_tag_dict[key]])

### Шаг 4. Тренировка модели.

Настало время натренировать нашу модель! 

Обучение модели будет происходить несколькими итерациями. При запуске тренировки трейнер вернет нам первую итерацию. Пока обучение не будет закончено, метод, возвращающий итерацию будет возвращать статус "training".

In [14]:
import time
print("training")

iteration = trainer.train_project(project.id)

while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print("training status: " + iteration.status)
    time.sleep(1)
    
trainer.update_iteration(project.id, iteration.id, is_default=True)

training
training status: Training
training status: Training
training status: Training
training status: Training
training status: Training
training status: Training
training status: Training
training status: Training
training status: Training
training status: Completed


### Шаг 5. Работа с обученной моделью.

Теперь, обучив модель, попробуем дать ей на вход какой-нибудь пример, который она должна отгадать. 

#### Чемодан

In [16]:
suitcase_url = "https://images.ua.prom.st/409496703_w640_h640_aw8j0hfwwto.jpg"

results = predictor.predict_image_url(project.id, iteration.id, url=suitcase_url)

for prediction in results.predictions:
        print("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

	Suitcase: 91.85%
	Bag: 35.47%
	Briefcase: 25.86%
	Handbag: 2.52%
	Backpack: 0.56%


#### Рюкзак

In [17]:
backpack_url = "http://garsingshop.by/pics/catalog/product/img/1121_img.JPG"

results = predictor.predict_image_url(project.id, iteration.id, url=backpack_url)

for prediction in results.predictions:
        print("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

	Backpack: 87.00%
	Suitcase: 11.80%
	Bag: 8.05%
	Handbag: 3.28%
	Briefcase: 0.03%


#### Сумка

In [18]:
bag_url = "https://www.bodyboss.ru/upload/resize_cache/iblock/19e/1110_624_2/19e972d7ab8bdad21511b9db30a2e7e0.jpg"

results = predictor.predict_image_url(project.id, iteration.id, url=bag_url)

for prediction in results.predictions:
        print("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

	Bag: 80.83%
	Backpack: 78.47%
	Suitcase: 7.83%
	Handbag: 6.94%
	Briefcase: 0.92%


#### Ручная сумка

In [19]:
handbag_url = "http://tokishindenis.ru/upload_modules/goods/goods/full/c614eeeace923537f082342514ab0c63.JPG"

results = predictor.predict_image_url(project.id, iteration.id, url=handbag_url)

for prediction in results.predictions:
        print("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

	Handbag: 93.89%
	Bag: 35.08%
	Briefcase: 1.03%
	Suitcase: 0.07%
	Backpack: 0.03%


#### Портфели

In [22]:
briefcase_url = "https://s11.stc.all.kpcdn.net/share/i/12/10181815/inx960x640.jpg"

results = predictor.predict_image_url(project.id, iteration.id, url=briefcase_url)

for prediction in results.predictions:
        print("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

	Suitcase: 32.32%
	Handbag: 26.20%
	Briefcase: 22.34%
	Bag: 4.23%
	Backpack: 1.30%
